<a href="https://colab.research.google.com/github/Tolulope-Akinmoju/AdventureWorks-PowerBI-Sales-Dashboard/blob/main/HR_AdvML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import statsmodels.api as sm
from statsmodels.formula.api import ols

import patsy
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model







In [ ]:

##Import Data
data= "/content/ClickedMSHR_MachineLearning.csv"
#column_names = ['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education',
     #'EducationField' , 'EmployeeCount', 'EmployeeID', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus',
    # 'MonthlyIncome', 'NumCompaniesWorked', 'Over18', 'PercentSalaryHike', 'StandardHours',
     #'StockOptionLevel','TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
    # 'YearsWithCurrManager']#
raw_dataset = pd.read_csv(data)

In [ ]:
dataset =raw_dataset.copy()
dataset.head()

Check for NA Values ad Dtrop if Need Be


In [ ]:
dataset.isna().sum()

Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

In [ ]:
dataset = dataset.dropna()

One hot encoding

In [ ]:
#binary encoding
dataset['Overtime'] = dataset['Overtime'].map(['No' : 0, 'Yes' :1})
dataset['Gender'] = dataset['Gender'].map(['Male' : 0, 'Female' :1})
dataset['Over18'] = dataset['Over18'].map(['No' : 0, 'Y' :1}

In [ ]:
#encode categorical columns which are ordinal
from sklearn.preprocessing import LabelEncoder
encoding_cols = ['BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus']
label_encoders = {}
for column i encoding_cols:
  label_encoders[column] = LabelEncoder()
  dataset[column] = label_encoders[column]. fit_transform(dataset[column])

In [ ]:
dataset = head()

In [ ]:
#create category maps

dataset['Education'] = dataset['Education'].map({1: 'Below College', 2: 'College', 3: 'Bachelor', 4: 'Master', 5: 'Doctor'})
dataset['JobLevel'] = dataset['JobLevel'].map({1: 'Entry', 2: 'Mid', 3: 'Senior', 4: 'Expert', 5: 'Executive'})
dataset['StockOptionLevel'] = dataset['StockOptionLevel'].map({0: 'Low', 1: 'Medium', 2: 'High', 3: 'Very High'})

In [ ]:
#Onehot encode variables
dataset = pd.get_dummies(dataset, columns= ['Education', 'JobLevel', 'StockOptionLevel'], prefix='', prefix_sep= '')

In [ ]:
#Onehot encode variables
dataset = pd.get_dummies(dataset, columns= ['Attrition'], prefix='', prefix_sep= '')

Create Training and Test Dataset

In [ ]:
train_dataset = dataset.copy
train_dataset = train_dataset()
test_dataset = dataset.copy
test_dataset = test_dataset()


Inspect data


In [ ]:
#Check for non-numerical information and convert categorical data
check_non_numerical_columns = [train_dataset, test_dataset]
def check_non_numerical_columns(datasets):
        for dataset in datasets:
          for column in dataset.columns:
            if train_dataset[column].dtype == 'object':
              print(f"Non-numerical data found in column: {column}")
              print(train_dataset[column].unique()) # Added and will test
              print()

In [ ]:
dataset.head()

MonthlyIncome and ANOVA


In [ ]:
#create train_anova dataset to ensure data integrity
def train_anova(train_dataset):
    anova_data = train_dataset[['MonthlyIncome', 'Attrition']].copy()
    return anova_data

anova_data = train_anova(train_dataset)

In [ ]:
#Fit an ANOVA model to determine if strong relationship exist btw MonthlyIncome and Attrition
anova_data = train_anova(train_dataset)
anova_model = ols('Attrition ~ M(MonthlyIncome)', data = anova_data).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=1)

#Print the ANOVA table
print(anova_table)

#Drop MonthlyIncome from datasets
train_dataset = train_dataset.drop(columns = ['MonthlyIncome'])
test_dataset = test_dataset.drop(columns=['MonthlyIncome'])

Build Decision Tree and Logistic Regression

In [ ]:
#Preprocess data
X=train_dataset.drop('Age', axis =1)
y= train_dataset['Age']

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state =42)

In [ ]:
#Create and Train Tree
clf = DecisionTreeClassifier(max_depth =4) #Customize parameters later (e.g., max_depth)
clf = clf.fit(X_train, y_train)


In [ ]:
#visualize results
plt.figure(figure =(20,10)) #Adjust figsize if needed
tree.plot_tree(clf, feature_names= X_train.columns, filled = True)
plt.show()